In [2]:
import DataProcessing
from pathlib import Path
import numpy as np
from tqdm import tqdm
import pptk
import math


In [3]:
pcd_tup = DataProcessing.load_from_ptx([Path('../../PatrickData/SongoMnara/10.33.47.ptx')])

File: 10.33.47.ptx
Rows: 5056
Cols: 2150
Total points: 10870400



In [4]:
pcd = np.hstack((pcd_tup[0], pcd_tup[1][:, None], pcd_tup[2][:, 1:], pcd_tup[2][:, 0][:, None]))
pcd.shape

(9015171, 7)

In [5]:
# pcd_dtype = [('x', np.float64),('y', np.float64),('z', np.float64),('i', np.float64),('g', np.float64),('b', np.float64),('class', np.float64),]
# pcd.dtype = pcd_dtype

In [6]:
pcd_copy = pcd.copy()

In [7]:
pcd = pcd_copy.copy()

In [11]:
split_indices = np.arange(0, pcd.shape[0], pcd.shape[0] // 10)[1:]

In [1]:
pcd.view('f8,f8,f8,f8,f8,f8,f8').sort(order=['f0'], axis=0)
pcd = np.array_split(pcd, split_indices)

NameError: name 'pcd' is not defined

In [14]:
pcd[0].shape

(901517, 7)

In [15]:

for i, col in enumerate(tqdm(pcd)):
    col = col.view('f8,f8,f8,f8,f8,f8,f8').sort(order=['f1'], axis=1)

100%|██████████| 11/11 [00:00<00:00, 51.99it/s]


In [16]:
pcd = np.vstack((pcd))

In [53]:
# DataProcessing.is_increasing(pcd[:,0])

True

In [54]:
# DataProcessing.is_increasing(pcd[:,1])

False

In [17]:

grid_mask = np.repeat(np.arange(0, 10, 1), math.ceil(9015171 / 10))[:pcd.shape[0], None]

In [73]:
pcd = np.hstack((pcd, grid_mask))

In [ ]:
num_points = 1024
num_cells = 10
value_range = 100
a = np.random.randint(0,value_range,(num_points,3))

total_distances = a.max(axis=0)-a.min(axis=0)
intervals = np.asarray([(a.min(axis=0) + total_distances//num_cells*i) for i in range(num_cells)])

a.view('i8,i8,i8').sort(order=['f0'], axis=0)
interval_idxs_x = [DataProcessing.find_nearest_id(a[:, 0], v) for v in intervals[:-1,0]]


# split_indices = np.arange(0, a.shape[0], a.shape[0] // num_cells)[1:]

# grid_mask = np.repeat(np.arange(0, num_cells, 1), math.ceil(num_points / num_cells))[:num_points]

a = np.array_split(a, split_indices)
for i, col in enumerate(tqdm(a)):
    col = col.view('i8,i8,i8').sort(order=['f1'], axis=0)
a = np.vstack(a)

a = np.hstack((a,grid_mask[:,None]))

v=pptk.viewer(a[:,:3],a[:,3])
v.set(point_size=.1)